# Модели стохастических объектов (методы анализа данных)
## Практическая работа №2
### КИ18-16 Прекель В.А.
### Вариант 14.
### Признаки `cat6_brandOfSoaps`, `cat2_hairColor`, `num1_15`, `num2_15`, `num3_15`.

In [1]:
from collections import Counter
from collections import Counter
from scipy.stats import sigmaclip
from sklearn import preprocessing
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import scipy as scp
import seaborn as sns
import seaborn as ssn

In [3]:
all_data = pd.read_excel('practice_dataframe.xlsx')

data = pd.DataFrame({
    'cat6_brandOfSoaps': all_data['cat6_brandOfSoaps'],
    'cat2_hairColor': all_data['cat2_hairColor'],
    'num1_15': all_data['num1_15'],
    'num2_15': all_data['num2_15'],
    'num3_15': all_data['num3_15']
})
data

,cat6_brandOfSoaps,cat2_hairColor,num1_15,num2_15,num3_15
0,lux,black,282.228024,NaN,807.976365
1,NaN,Red,1039.089956,268.388947,444.529442
2,Lux,black,500.696085,174.992127,428.723966
3,lux,Red,1346.429995,108.647900,520.238037
4,Dove,black,230.576445,150.113065,896.476167
...,...,...,...,...,...
995,Dove,Red,422.732662,170.810692,586.254084
996,lux,Brown,783.565442,237.086397,826.693312
997,Dove,Red,410.595523,196.686858,600.377791
998,Lux,black,-237.136035,179.262913,371.380055


In [4]:
data.describe()

,num1_15,num2_15
count,1000.000000,950.000000
mean,312.524662,148.963833
std,2443.714313,106.847552
min,-9999.000000,-184.922189
25%,246.738757,82.347973
50%,814.713260,145.220453
75%,1284.983740,215.636532
max,3201.191252,479.162302


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cat6_brandOfSoaps  980 non-null    object 
 1   cat2_hairColor     980 non-null    object 
 2   num1_15            1000 non-null   float64
 3   num2_15            950 non-null    float64
 4   num3_15            971 non-null    object 
dtypes: float64(2), object(3)
memory usage: 39.2+ KB
